In [2]:
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from itertools import combinations_with_replacement
from itertools import combinations
from itertools import permutations
from scipy.stats import qmc

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
A = np.array([[4,3,0],
              [0,0,0]])
print(A[np.nonzero(A)])

[4 3]


In [11]:
np.random.seed(9)
Array_1 = np.random.rand(3,2)
Array_2 = np.random.rand(3,2)

print(Array_1)
print(Array_2)

ds = gdal.Open('image.tif')
# loop through each band

path = 

with open("converted.raw", "wb") as outfile:
    for bi in range(ds.RasterCount):
        band = ds.GetRasterBand(bi + 1)
        # Read this band into a 2D NumPy array
        ar = band.ReadAsArray()
        print('Band %d has type %s'%(bi + 1, ar.dtype))   
        np.save(outfile, ar.astype('uint16'))

[[0.01037415 0.50187459]
 [0.49577329 0.13382953]
 [0.14211109 0.21855868]]
[[0.41850818 0.24810117]
 [0.08405965 0.34549864]
 [0.16677635 0.87855909]]
